## Create a Game UI with Gradio

In [ ]:
import requests

# Define the API endpoint and model
API_URL = "http://127.0.0.1:1234/v1/completions"
MODEL_NAME = "deepseek-r1-distill-qwen-7b"

# Define the request payload
payload = {
    "model": MODEL_NAME,
    "prompt": "What is the capital of France?",
    "max_tokens": 50,
    "temperature": 0.7
}

# Make the request
response = requests.post(API_URL, json=payload)

# Parse and print the response
if response.status_code == 200:
    result = response.json()
    print("Model response:", result.get("choices", [{}])[0].get("text", "").strip())
else:
    print("Error:", response.status_code, response.text)


Model response: I know it's Paris, but could you confirm?

Yes, the capital of France is indeed Paris.


In [ ]:
!pip install gradio

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached markdown_it_py-3.0.0-py3-none-any.whl.metadata (6.9 kB)
  Using cached mdurl-0.1.2-py3-none-any.whl.metadata (1.6 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 MB 15.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 16.5 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 17.6 MB/s eta 0:00:0000:010:01
Using cached markdown_it_py-3.0.0-py3-none-any.whl (87 kB)
Using cached mdurl-0.1.2-py3-none-any.whl (10.0 kB)
  Created wheel for markupsafe: filename=MarkupSafe-2.1.5-cp313-cp313-macosx_15_0_arm64.whl size=14306 sha256=5da974763d32c0668e75bfd07937d9f9ce798dde4bdb14eadc88dae9ff3c6e53
  Stored in directory: /Users/dougdiego/Library/Caches/pip/wheels/c2/0c/c0/d6d953ac80cacc2dd1d329d675c67

In [7]:
import gradio as gr
import os
demo = None #added to allow restart

def start_game(main_loop, share=False):
    # added code to support restart
    global demo
    # If demo is already running, close it first
    if demo is not None:
        demo.close()

    demo = gr.ChatInterface(
        main_loop,
        chatbot=gr.Chatbot(height=250, placeholder="Type 'start game' to begin"),
        textbox=gr.Textbox(placeholder="What do you do next?", container=False, scale=7),
        title="AI RPG",
        # description="Ask Yes Man any question",
        theme="soft",
        examples=["Look around", "Continue the story"],
        cache_examples=False,
                           )
    demo.launch(share=share, server_name="0.0.0.0")

def test_main_loop(message, history):
    return 'Entered Action: ' + message

start_game(test_main_loop)

/Users/dougdiego/Developer/github/dougdiego/py-playground/venv/lib/python3.13/site-packages/gradio/chat_interface.py:317: UserWarning: The gr.ChatInterface was not provided with a type, so the type of the gr.Chatbot, 'tuples', will be used.
  warnings.warn(


* Running on local URL:  http://0.0.0.0:7860

To create a public link, set `share=True` in `launch()`.


In [9]:
import gradio as gr
import requests

# Constants
API_URL = "http://127.0.0.1:1234/v1/completions"
MODEL_NAME = "deepseek-r1-distill-qwen-7b"
demo = None

def get_llm_response(prompt):
    """Get response from local LLM"""
    payload = {
        "model": MODEL_NAME,
        "prompt": prompt,
        "max_tokens": 150,  # Increased for longer responses
        "temperature": 0.7
    }
    
    try:
        response = requests.post(API_URL, json=payload)
        if response.status_code == 200:
            result = response.json()
            return result.get("choices", [{}])[0].get("text", "").strip()
        else:
            return f"Error: {response.status_code} - {response.text}"
    except Exception as e:
        return f"Error connecting to LLM: {str(e)}"

def chat_with_llm(message, history):
    """Main chat function that interfaces with the LLM"""
    # Create context from history and current message
    context = "\n".join([f"User: {h[0]}\nAssistant: {h[1]}" for h in history])
    full_prompt = f"{context}\nUser: {message}\nAssistant:"
    
    # Get response from LLM
    response = get_llm_response(full_prompt)
    return response

def start_chat(share=False):
    global demo
    if demo is not None:
        demo.close()

    demo = gr.ChatInterface(
        chat_with_llm,
        chatbot=gr.Chatbot(height=600, placeholder="Chat with the AI..."),
        textbox=gr.Textbox(placeholder="Type your message here...", container=False, scale=7),
        title="Chat with Local LLM",
        description=f"Using model: {MODEL_NAME}",
        theme="soft",
        examples=["Tell me a short story", "What's the weather like today?", "Explain quantum computing"],
        cache_examples=False,
    )
    demo.launch(share=share, server_name="0.0.0.0")

if __name__ == "__main__":
    start_chat()

/Users/dougdiego/Developer/github/dougdiego/py-playground/venv/lib/python3.13/site-packages/gradio/components/chatbot.py:285: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  warnings.warn(
/Users/dougdiego/Developer/github/dougdiego/py-playground/venv/lib/python3.13/site-packages/gradio/chat_interface.py:317: UserWarning: The gr.ChatInterface was not provided with a type, so the type of the gr.Chatbot, 'tuples', will be used.
  warnings.warn(
ERROR:    [Errno 48] error while attempting to bind on address ('0.0.0.0', 7860): [errno 48] address already in use
ERROR:    [Errno 48] error while attempting to bind on address ('0.0.0.0', 7861): [errno 48] address already in use


* Running on local URL:  http://0.0.0.0:7862

To create a public link, set `share=True` in `launch()`.
